In [13]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [14]:
import os
HOME = os.getcwd()
print(HOME)

/content/yolov9


In [15]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 147 (delta 30), reused 28 (delta 28), pack-reused 105
Receiving objects: 100% (147/147), 607.49 KiB | 17.36 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/yolov9/yolov9


In [16]:
!pip install -q roboflow

In [17]:
import roboflow

from IPython.display import Image

In [18]:
!mkdir -p {HOME}/weights

In [19]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [20]:
%cd {HOME}/yolov9
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WMBOqkEzTUre6KzfvX45")
project = rf.workspace("objectdetection-xpbt7").project("underwater-object-detection-mcjr0")
dataset = project.version(8).download("yolov9")


/content/yolov9/yolov9
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Underwater-Object-Detection-8 in yolov9:: 100%|██████████| 2374/2374 [00:00<00:00, 7086.22it/s]


In [24]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 100 --img 640 --device 1 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

/content/yolov9/yolov9
2024-02-28 17:38:32.296368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 17:38:32.296455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 17:38:32.300158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 17:38:33.811814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=/content/yolov9/weights/gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=/content/yolov9/yolov9/Underwater-Object-Detection-8/data.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=16, imgsz=640, r

In [22]:
!ls {HOME}/yolov9/runs/train/exp/

ls: cannot access '/content/yolov9/yolov9/runs/train/exp/': No such file or directory


In [23]:
Image(filename=f"{HOME}/yolov9/runs/train/exp/results.png", width=1000)

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov9/yolov9/runs/train/exp/results.png'

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp/confusion_matrix.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp/val_batch0_pred.jpg", width=1000)

In [ ]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

In [ ]:
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {dataset.location}/valid/images